In [ ]:
import cv2
import numpy as np
import moviepy.editor as mp
import ffmpeg

def denoise_frame(frame):
    """Riduce il rumore usando il filtro Gaussiano."""
    return cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)

def enhance_contrast(frame):
    """Migliora il contrasto usando la equalizzazione dell'istogramma."""
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def restore_video(input_path, output_path):
    """Elabora e ripulisce il video."""
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, 
                          (int(cap.get(3)), int(cap.get(4))))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = denoise_frame(frame)  # Riduzione rumore
        frame = enhance_contrast(frame)  # Miglioramento contrasto
        
        out.write(frame)
        cv2.imshow('Restoring...', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def improve_audio(input_video, output_video):
    """Migliora l'audio del video usando FFmpeg."""
    input_audio = "temp_audio.wav"
    output_audio = "temp_audio_enhanced.wav"
    
    # Estrai audio
    ffmpeg.input(input_video).output(input_audio, acodec='pcm_s16le').run()
    
    # Migliora audio (puoi sostituire con un processo avanzato)
    ffmpeg.input(input_audio).output(output_audio, af='volume=1.5').run()
    
    # Reincorpora audio migliorato
    ffmpeg.input(output_video).input(output_audio).output("final_video.mp4", vcodec='copy', acodec='aac').run()

def main():
    input_video = 'old_video.mp4'  # Sostituisci con il tuo file
    processed_video = 'restored_video.mp4'
    
    print("Restauro in corso...")
    restore_video(input_video, processed_video)
    
    print("Miglioramento audio...")
    improve_audio(input_video, processed_video)
    
    print("Restauro completato! Video salvato come final_video.mp4")

if __name__ == "__main__":
    main()

# Approcio con AI 

In [ ]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
import moviepy.editor as mp
import ffmpeg
from realesrgan import RealESRGAN

def load_model():
    """Carica il modello AI di super-risoluzione."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RealESRGAN(device, scale=4)
    model.load_weights('weights/RealESRGAN_x4.pth')
    return model

def enhance_frame(frame, model):
    """Migliora il frame con AI Super-Resolution."""
    transform = transforms.ToTensor()
    frame = transform(frame).unsqueeze(0).to(model.device)
    with torch.no_grad():
        enhanced_frame = model(frame).squeeze(0).cpu().numpy()
    enhanced_frame = np.clip(enhanced_frame * 255, 0, 255).astype(np.uint8)
    return enhanced_frame.transpose(1, 2, 0)

def restore_video(input_path, output_path, model):
    """Elabora il video frame per frame usando AI."""
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, 
                          (int(cap.get(3)) * 4, int(cap.get(4)) * 4))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = enhance_frame(frame, model)  # Super-Risoluzione
        out.write(frame)
        cv2.imshow('Restoring AI...', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()

def improve_audio(input_video, output_video):
    """Migliora l'audio del video usando FFmpeg."""
    input_audio = "temp_audio.wav"
    output_audio = "temp_audio_enhanced.wav"
    
    # Estrai audio
    ffmpeg.input(input_video).output(input_audio, acodec='pcm_s16le').run()
    
    # Migliora audio (puoi sostituire con un processo avanzato)
    ffmpeg.input(input_audio).output(output_audio, af='volume=1.5').run()
    
    # Reincorpora audio migliorato
    ffmpeg.input(output_video).input(output_audio).output("final_video.mp4", vcodec='copy', acodec='aac').run()

def main():
    input_video = 'old_video.mp4'  # Sostituisci con il tuo file
    processed_video = 'restored_video.mp4'
    
    print("Caricamento modello AI...")
    model = load_model()
    
    print("Restauro AI in corso...")
    restore_video(input_video, processed_video, model)
    
    print("Miglioramento audio...")
    improve_audio(input_video, processed_video)
    
    print("Restauro completato! Video salvato come final_video.mp4")

if __name__ == "__main__":
    main()


https://github.com/ai-forever/Real-ESRGAN in caso non funzioni 